In [ ]:
import random
import math
import pygame

pygame.init()
pygame.mixer.init()

# Load siren sound
siren_sound = pygame.mixer.Sound("siren.wav")
siren_channel = pygame.mixer.Channel(1)
siren_channel.play(siren_sound, loops=1)

WIDTH, HEIGHT = 1000, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Iron Dome Defense System Simulation")

# Colors
RED = (255, 0, 0)
GREEN = (0, 255, 0)
ORANGE = (255, 165, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)

# Load Iron Dome image
iron_dome_image = pygame.image.load("iron_dome.jpg")
iron_dome_image = pygame.transform.scale(iron_dome_image, (120, 120))

class Explosion:
    def __init__(self, position):
        self.position = position
        self.radius = 1
        self.alpha = 255
        self.growth = 2
        self.fade_speed = 15
        
    def update(self):
        self.radius += self.growth
        self.alpha -= self.fade_speed
        if self.alpha < 0:
            self.alpha = 0
            
    def draw(self, surface):
        if self.alpha > 0:
            explosion_surf = pygame.Surface((self.radius * 2, self.radius * 2), pygame.SRCALPHA)
            pygame.draw.circle(explosion_surf, (255, 140, 0, self.alpha), (self.radius, self.radius), self.radius)
            surface.blit(explosion_surf, (self.position[0] - self.radius, self.position[1] - self.radius))
            
    def is_finished(self):
        return self.alpha <= 0

class Rocket:
    def __init__(self, id, launch_position, target_position, speed):
        self.id = id
        self.position = list(launch_position)
        self.target_position = target_position
        self.speed = speed
        self.destroyed = False
        self.hypersonic = False
        
    def move(self):
        direction = (self.target_position[0] - self.position[0], self.target_position[1] - self.position[1])
        dist = math.hypot(*direction)
        if dist == 0: return
        direction = (direction[0] / dist, direction[1] / dist)
        self.position[0] += direction[0] * self.speed
        self.position[1] += direction[1] * self.speed

class HypersonicRocket(Rocket):
    def __init__(self, id, launch_position, target_position):
        super().__init__(id, launch_position, target_position, speed=random.uniform(25, 30))
        self.hypersonic = True

class Interceptor:
    def __init__(self, launch_position, speed, target, explosions):
        self.position = list(launch_position)
        self.speed = speed
        self.target = target
        self.engaged = False
        self.explosions = explosions
    
    def move(self):
        if self.engaged: return
        direction = (self.target.position[0] - self.position[0], self.target.position[1] - self.position[1])
        dist = math.hypot(*direction)
        if dist == 0: return
        direction = (direction[0] / dist, direction[1] / dist)
        self.position[0] += direction[0] * self.speed
        self.position[1] += direction[1] * self.speed
        
    def check_interception(self):
        dist = math.hypot(self.position[0] - self.target.position[0], self.position[1] - self.target.position[1])
        chance = 0.5 if not self.target.hypersonic else 0.3
        if dist < 5.0:
            self.explosions.append(Explosion(self.target.position[:]))
            if random.random() < chance:
                self.target.destroyed = True
                print(f"✅ Interceptor destroyed Rocket {self.target.id}")
            else:
                print(f"⚠️ Interceptor missed Rocket {self.target.id}")
                self.target.destroyed = True
            self.engaged = True
            
class IronDomeSystem:
    def __init__(self):
        self.siren_playing = False
        self.rockets = []
        self.interceptors = []
        self.explosions = []
        self.protected_areas = [(WIDTH // 3, HEIGHT - 100), (2 * WIDTH // 3, HEIGHT - 100)]
        self.engaged_rockets = []
        self.dome_damage = [0, 0]
        self.detection_radius = 150
        self.disclaimer_x = WIDTH
        self.disclaimer_x2 = None